In [1]:
import os
import cv2
import torch
import torchvision
import numpy as np
from data_unpacker import video_packer

In [2]:
device = torch.device('cuda:0')

In [3]:
torch.cuda.get_device_name(0)

'GeForce GTX 1050'

In [5]:
model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

AttributeError: module 'torchvision.models.detection' has no attribute 'fasterrcnn_mobilenet_v3_large_fpn'

In [11]:
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, 2)
model_path = '/home/stasvlad/Documents/hse/tips&tricks/state_dict_model.pt'
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [12]:
model.to(device)
model.eval();

In [20]:
dummy_input = torch.randn(1, 3, 720, 1280, dtype=torch.float).to(device)
# INIT LOGGERS
starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
repetitions = 30
timings=np.zeros((repetitions,1))
#GPU-WARM-UP
for _ in range(10):
    _ = model(dummy_input)
# MEASURE PERFORMANCE
with torch.no_grad():
    for rep in range(repetitions):
        starter.record()
        _ = model(dummy_input)
        ender.record()
        # WAIT FOR GPU SYNC
        torch.cuda.synchronize()
        curr_time = starter.elapsed_time(ender)
        timings[rep] = curr_time
mean_syn = np.sum(timings) / repetitions
std_syn = np.std(timings)
print(mean_syn)

383.0539154052734


In [15]:
1000 / 30

33.333333333333336